In [11]:
SAMPLE_TEXT = """
6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0

fold along y=7
fold along x=5
"""

In [12]:
def tokenize_line(line):
    if line.startswith("fold"):
        xy, position = line.split(" ")[-1].split("=")
        return "fold", xy, int(position)
    else:
        x, y = line.split(",")
        return "point", int(x), int(y)


def parse_text(raw_text):
    return [tokenize_line(l) for l in raw_text.split("\n") if l]


def read_input():
    with open("input.txt", "rt") as f:
        return f.read()

def split_by_type(lines):
    points = (l for l in lines if l[0] == 'point')
    folds = (l for l in lines if l[0] == 'fold')
    return list(points), list(folds)

In [35]:
class Board:
    def __init__(self, points):
        self.rows = []
        self.max_x, self.max_y = self.get_limits(points)
        self.create_board(points)

    def create_board(self, points):
        for y in range(self.max_y + 1):
            self.rows.append([None] * (self.max_x + 1))
        for _, x, y in points:
            self.mark_point(x, y, '#')

    def get_limits(self, points):
        max_x = 0
        max_y = 0
        for _, x, y in points:
            max_x = max(max_x, x)
            max_y = max(max_y, y)
        if max_x % 2 != 0:
            max_x += 1
        if max_y % 2 != 0:
            max_y += 1
        return max_x, max_y

    def get_value(self, x, y):
        return self.rows[y][x]

    def mark_point(self, x, y, value):
        if value is None:
            return
        self.rows[y][x] = value

    def clear_point(self, x, y):
        self.rows[y][x] = None

    def fold(self, fold):
        _, xy, position = fold
        if xy == 'x':
            if position != self.max_x // 2:
                raise ValueError(f"Can't fold along x={position}, should be {self.max_x // 2}")
            self.fold_x(position)
        elif xy == 'y':
            if position != self.max_y // 2:
                raise ValueError(f"Can't fold along y={position}, should be {self.max_y // 2}")
            self.fold_y(position)
        else:
            raise ValueError(f"Invalid fold: {xy}")

    def fold_x(self, position):
        copy_to = list(range(position))
        copy_from = list(range(self.max_x, position, -1))
        copy_to_from = list(zip(copy_to, copy_from))
        # print('x', position, self.max_x, self.max_y)
        for y in range(self.max_y + 1):
            for x_new, x_old in copy_to_from:
                self.mark_point(x_new, y, self.get_value(x_old, y))
                self.clear_point(x_old, y)
        self.max_x = position - 1
        # print('x', position, self.max_x, self.max_y)

    def fold_y(self, position):
        copy_to = list(range(position))
        copy_from = list(range(self.max_y, position, -1))
        copy_to_from = list(zip(copy_to, copy_from))
        # print('y', position, self.max_x, self.max_y)
        for x in range(self.max_x + 1):
            for y_new, y_old in copy_to_from:
                self.mark_point(x, y_new, self.get_value(x, y_old))
                self.clear_point(x, y_old)
        self.max_y = position - 1
        # print('y', position, self.max_x, self.max_y)

    def count_points(self):
        points = 0
        for x in range(self.max_x + 1):
            for y in range(self.max_y + 1):
                if self.get_value(x, y):
                    points +=1
        return points

    def do_folds(self, folds):
        for f in folds:
            self.fold(f)

    def __str__(self):
        result = f"Board: max_x={self.max_x}, max_y={self.max_y}\n"
        for y in range(self.max_y + 1):
            for x in range(self.max_x + 1):
                v = self.get_value(x, y)
                result += v if v else "."
            result += "\n"
        return result

In [36]:
# Didn't love the approach above. Trying with a dictionary of points rather than arrays.
class SparseBoard:
    def __init__(self, points):
        self.points = {}
        self.max_x = 0
        self.max_y = 0
        for _, x, y in points:
            self.points[x, y] = '#'
            self.max_x = max(self.max_x, x)
            self.max_y = max(self.max_y, y)

    def count_points(self):
        return len(self.points)

    def fold(self, fold):
        _, direction, position = fold
        if direction == 'x':
            offset = lambda x, y: (position * 2 - x, y)
            x_limit = position - 1
            y_limit = self.max_y
        else:
            offset = lambda x, y: (x, position * 2 - y)
            x_limit = self.max_x
            y_limit = position - 1

        for x in range(x_limit + 1):
            for y in range(y_limit + 1):
                o = offset(x, y)
                if o in self.points:
                    self.points[x, y] = self.points.get((x, y)) or self.points[o]
                    del self.points[o]

        self.max_x, self.max_y = x_limit, y_limit

    def do_folds(self, folds):
        for f in folds:
            self.fold(f)

    def __str__(self):
        result = f"SparseBoard: max_x={self.max_x}, max_y={self.max_y}\n"
        for y in range(self.max_y + 1):
            for x in range(self.max_x + 1):
                result += self.points.get((x, y), '.')
            result += "\n"
        return result

In [37]:
points, folds = split_by_type(parse_text(SAMPLE_TEXT))
board = Board(points)
print(board)
board.do_folds(folds)
print(board)
board.count_points()

Board: max_x=10, max_y=14
...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
...........
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........

Board: max_x=4, max_y=6
#####
#...#
#...#
#...#
#####
.....
.....



16

In [38]:
points, folds = split_by_type(parse_text(SAMPLE_TEXT))
board = SparseBoard(points)
print(board)
board.do_folds(folds)
print(board)
board.count_points()

SparseBoard: max_x=10, max_y=14
...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
...........
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........

SparseBoard: max_x=4, max_y=6
#####
#...#
#...#
#...#
#####
.....
.....



16

In [39]:
# Just the first fold
points, folds = split_by_type(parse_text(read_input()))
board = Board(points)
board.fold(folds[0])
board.count_points()

684

In [40]:
# Just the first fold
points, folds = split_by_type(parse_text(read_input()))
board = SparseBoard(points)
board.fold(folds[0])
board.count_points()

684

In [41]:
# All folds
points, folds = split_by_type(parse_text(read_input()))
board = Board(points)
board.do_folds(folds)
print(board)

Board: max_x=39, max_y=5
..##.###..####.###..#.....##..#..#.#..#.
...#.#..#....#.#..#.#....#..#.#.#..#..#.
...#.#..#...#..###..#....#....##...####.
...#.###...#...#..#.#....#.##.#.#..#..#.
#..#.#.#..#....#..#.#....#..#.#.#..#..#.
.##..#..#.####.###..####..###.#..#.#..#.



In [42]:
# All folds with SparseBoard
points, folds = split_by_type(parse_text(read_input()))
board = SparseBoard(points)
board.do_folds(folds)
print(board)

SparseBoard: max_x=39, max_y=5
..##.###..####.###..#.....##..#..#.#..#.
...#.#..#....#.#..#.#....#..#.#.#..#..#.
...#.#..#...#..###..#....#....##...####.
...#.###...#...#..#.#....#.##.#.#..#..#.
#..#.#.#..#....#..#.#....#..#.#.#..#..#.
.##..#..#.####.###..####..###.#..#.#..#.



In [51]:
# Performance comparison
points, folds = split_by_type(parse_text(read_input()))
def run(board_type):
    board = board_type(points)
    board.do_folds(folds)

In [52]:
%timeit run(Board)

621 ms ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
%timeit run(SparseBoard)

380 ms ± 102 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
